In [378]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os

In [379]:
# Load environmental variables
load_dotenv(override=True)

# Assign raw data and plate diagram files from environment
RAW_SAMPLE_1 = os.getenv("RAW_SAMPLE_1")
RAW_SAMPLE_1_DIAGRAM = os.getenv("RAW_SAMPLE_1_DIAGRAM")

# Read in the raw data and plate diagram as pandas dataframes
df_data = pd.read_excel(RAW_SAMPLE_1, sheet_name=0)
df_diagram = pd.read_excel(RAW_SAMPLE_1_DIAGRAM)

df_diagram

,Unnamed: 0,1,2,3,4,5,6,7,8,9,10,11,12
0,A,NaN,D49,D49 dup**,D73,D73 dup,D89,D89 dup,D168,D168 dup,D185,D185 dup,NaN
1,B,NaN,D53,D53 dup,D76,D76 dup,D92,D92 dup,D169*,D169 dup,D192,D192 dup,NaN
2,C,NaN,D56,D56 dup,D77,D77 dup,D149,D149 dup,D172,D172 dup,D193,D193 dup,NaN
3,D,NaN,D57,D57 dup,D80,D80 dup,D152,D152 dup,D173,D173 dup,D200,D200 dup,NaN
4,E,NaN,D60,D60 dup,D81,D81 dup,D153,D153 dup,D176,D176 dup,D17,D17 dup,NaN
5,F,NaN,D65,D65 dup,D84,D84 dup,D157,D157 dup,D180,D180 dup,D49 dup,NaN,NaN
6,G,NaN,D68,D68 dup,D85,D85 dup,D160,D160 dup,D181,D181 dup,NaN,NaN,NaN
7,H,NaN,D72,D72 dup,D88,D88 dup,D165,D165 dup,D184,D184 dup,NaN,NaN,NaN


In [380]:
# Set the first column as the index, remove whitespace and add a space to the "dup" values
df_diagram = df_diagram.set_index(df_diagram.columns[0])
df_diagram = df_diagram.replace('\s+', '', regex=True)
df_diagram = df_diagram.replace('dup', ' dup', regex=True)

# Use the rows and columns (besides the first one) of the plate diagram to create a dictionary of corresponding Sample and Well IDs
sample_map = {}

for row in df_diagram.index:
    for col in df_diagram.columns[1:]:
        well_id = f"{row}{int(col):02d}"
        sample_name = df_diagram.loc[row, col]
        sample_map[well_id] = sample_name

# Read in the raw qPCR data and map the well IDs to sample names using the dictionary
df_data["Sample"] = df_data["Well"].map(sample_map)

df_data

,Unnamed: 0,Well,Fluor,Target,Content,Sample,Biological Set Name,Cq,Cq Mean,Cq Std. Dev,Starting Quantity (SQ),Log Starting Quantity,SQ Mean,SQ Std. Dev,Set Point,Well Note
0,NaN,A01,SYBR,NaN,Unkn,NaN,NaN,NaN,0.000000,0,NaN,NaN,0.0,0,72,NaN
1,NaN,A02,SYBR,NaN,Unkn,D49,NaN,19.214765,19.214765,0,NaN,NaN,NaN,0,72,NaN
2,NaN,A03,SYBR,NaN,Unkn,D49 dup**,NaN,19.586924,19.586924,0,NaN,NaN,NaN,0,72,NaN
3,NaN,A04,SYBR,NaN,Unkn,D73,NaN,19.634851,19.634851,0,NaN,NaN,NaN,0,72,NaN
4,NaN,A05,SYBR,NaN,Unkn,D73 dup,NaN,19.645373,19.645373,0,NaN,NaN,NaN,0,72,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,NaN,H08,SYBR,NaN,Unkn,D184,NaN,19.837336,19.837336,0,NaN,NaN,NaN,0,72,NaN
92,NaN,H09,SYBR,NaN,Unkn,D184 dup,NaN,19.822316,19.822316,0,NaN,NaN,NaN,0,72,NaN
93,NaN,H10,SYBR,NaN,Unkn,NaN,NaN,NaN,0.000000,0,NaN,NaN,0.0,0,72,NaN
94,NaN,H11,SYBR,NaN,Unkn,NaN,NaN,NaN,0.000000,0,NaN,NaN,0.0,0,72,NaN


In [381]:
# Select relevant columns, make mtDNA1 & mtDNA2 columns, and drop NaN rows
df = df_data[['Well', 'Cq', 'Sample']]

df['mtDNA1'] = "mtDNA1"
df['mtDNA2'] = "mtDNA2"

df = df.loc[:,["Well", "Sample", "mtDNA1", "mtDNA2", "Cq"]]

df = df.dropna()

C:\Users\Ivan Korostenskij\AppData\Local\Temp\ipykernel_10012\239484953.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mtDNA1'] = "mtDNA1"
C:\Users\Ivan Korostenskij\AppData\Local\Temp\ipykernel_10012\239484953.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mtDNA2'] = "mtDNA2"


In [382]:
# set mtDNA1 and mtDNA2 values to Cq values by treating mtDNA1 as the Cq for the first sample and mtDNA2 as the Cq for the duplicate sample if it exists as "Sample dup"

# Note, exactly "Sample dup" is used to avoid matching "Sample dup **" or any additions to the name

for row, index in df.iterrows():
    df.loc[row, 'mtDNA1'] = df.loc[row, 'Cq']
    if df.loc[row, 'Sample'] + ' dup' in df['Sample'].values:
        df.loc[row, 'mtDNA2'] = df.loc[df['Sample'] == df.loc[row, 'Sample'] + ' dup', 'Cq'].values[0]
    else:
        df.loc[row, 'mtDNA2'] = np.NAN

df = df.drop(columns=['Cq'])
df = df.dropna()

# calculate standard deviation of each row
df['St.Dev'] = df[['mtDNA1', 'mtDNA2']].std(axis=1)

In [383]:
# Throw warnings for standard deviations greater than .22

for row, index in df.iterrows():
    if df.loc[row, 'St.Dev'] > .22:
        print(f"Warning: Standard deviation for {df.loc[row, 'Sample']} is {round(df.loc[row, 'St.Dev'],ndigits=3)} "
              f"(Sample 1: {round(df.loc[row, 'mtDNA1'],ndigits=3)} vs Sample 2: {round(df.loc[row, 'mtDNA2'], ndigits=2)})")

In [384]:
# Drop index, sort by St.Dev, and download the file
df = df.sort_values(by=['St.Dev'], ascending=False)
df = df.reset_index(drop=True)

df.to_excel("test.xlsx")